Let's build a financial portfolio by investing in FCEL, SHOP companies and Dow 30, S&P 500 ETFs.

In [2]:
!pip install pandas_datareader

In [3]:
#Import relevant libraries
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
#Specify the stocks. I take top trending stocks from yahoo finance and couple of ETFs to devirsify 
#(Specific stocks chosen just to demonstrate an example)

stock = ['FCEL', 'SHOP','^DJI','^GSPC']
data = web.DataReader(stock,data_source="yahoo",start='01/01/2019',end='12/01/2019')['Adj Close']

#Convert to percentage format
stock_return = data.pct_change()
stock_return.round(4)*100
stock_return.head()

Symbols,FCEL,SHOP,^DJI,^GSPC
Date,,,,
2019-01-02,NaN,NaN,NaN,NaN
2019-01-03,-0.035088,-0.056759,-0.028271,-0.024757
2019-01-04,0.018182,0.063718,0.032925,0.034336
2019-01-07,0.035714,0.045850,0.004190,0.007010
2019-01-08,0.017241,0.007272,0.010883,0.009695


In [25]:
# Mean returns and covariances
mean_returns = stock_return.mean()
cov_matrix = stock_return.cov()
print (mean_returns)
print (cov_matrix)

Symbols
FCEL    -0.000559
SHOP     0.004182
^DJI     0.000786
^GSPC    0.000966
dtype: float64
Symbols      FCEL      SHOP      ^DJI     ^GSPC
Symbols                                        
FCEL     0.024780  0.000049  0.000076  0.000075
SHOP     0.000049  0.000759  0.000081  0.000100
^DJI     0.000076  0.000081  0.000065  0.000063
^GSPC    0.000075  0.000100  0.000063  0.000066


In [28]:
#Monte Carlo Simulation

NumberOfIterations = 10000
simulation_results = np.zeros((4+len(stock)-1,NumberOfIterations))

for i in range(NumberOfIterations):
    
        #Random weights
        weights = np.array(np.random.random(4))
        weights /= np.sum(weights)
        
        #Return and STDV
        portfolio_return = np.sum(mean_returns * weights)
        portfolio_std_dev = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights)))

        simulation_results[0,i] = portfolio_return
        simulation_results[1,i] = portfolio_std_dev
        
        #Calculate Sharpe ratio and store it in the array
        #Sharpe ratio measures the performance of an investment compared to a risk-free asset, after adjusting for its risk. It is defined as the difference between the returns of the investment and the risk-free return, divided by the standard deviation of the investment. 
        #Larger the ratio the better
        simulation_results[2,i] = simulation_results[0,i] / simulation_results[1,i]
        
        
        #Save the weights
        for j in range(len(weights)):
                simulation_results[j+3,i] = weights[j]
                
#Create a dataframe
simulation_frame = pd.DataFrame(simulation_results.T,columns=['Return','Stdev','Sharpe',stock[0],stock[1],stock[2],
                                                              stock[3]])

print (simulation_frame.head ())
print (simulation_frame.tail ())

     Return     Stdev    Sharpe      FCEL      SHOP      ^DJI     ^GSPC
0  0.001751  0.011739  0.149129  0.022714  0.265220  0.188790  0.523275
1  0.000446  0.085081  0.005247  0.538320  0.102582  0.159679  0.199419
2  0.000811  0.062260  0.013023  0.391561  0.159224  0.390163  0.059053
3  0.000678  0.034868  0.019432  0.215347  0.031657  0.344711  0.408285
4  0.000857  0.048753  0.017577  0.304711  0.127871  0.309965  0.257453
        Return     Stdev    Sharpe      FCEL      SHOP      ^DJI     ^GSPC
9995  0.002397  0.018714  0.128083  0.066033  0.492309  0.288172  0.153486
9996  0.001840  0.044280  0.041563  0.268046  0.405456  0.117076  0.209422
9997  0.000939  0.066925  0.014030  0.421077  0.194586  0.060733  0.323604
9998  0.000721  0.055311  0.013028  0.347296  0.107718  0.346348  0.198638
9999  0.000248  0.067439  0.003671  0.425649  0.005253  0.479231  0.089868


In [29]:
#Find weights distributions which produce highest Sharpe Ratio

max_sharpe = simulation_frame.iloc[simulation_frame['Sharpe'].idxmax()]
max_sharpe

Return    0.002123
Stdev     0.013159
Sharpe    0.161298
FCEL      0.000996
SHOP      0.367906
^DJI      0.141811
^GSPC     0.489286
Name: 7987, dtype: float64

Based on the resutls, our investment would be primarily in S&P 500, Shopify and Dow 30